In [1]:
import re
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv1D, BatchNormalization, Activation
from keras.layers import Embedding, Input, Dense, Dropout, Lambda, MaxPooling1D
from keras.optimizers import SGD
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint

/home/mukesh/miniconda3/envs/statoil/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#len(ALPHABET)=68
ALPHABET = 'abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:’"/|_#$%ˆ&*˜‘+=<>()[]{} '
FEATURE_LEN = 1024 #maxlen
path = '../data/'
TRAIN_DATA_FILE=path+'train.csv'
TEST_DATA_FILE=path+'test.csv'

In [3]:
def get_char_dict():
    char_dict={}
    for i,c in enumerate(ALPHABET):
        char_dict[c]=i+1
    return char_dict

def char2vec(text, max_length=FEATURE_LEN):
    char_dict = get_char_dict()
    data=np.zeros(max_length)
    
    for i in range(0, len(text)):
        if i >= max_length:
            return data
        
        elif text[i] in char_dict:
            data[i] = char_dict[text[i]]
        
        else:
            data[i]=68
    return data
    

def conv_shape(conv):
    return conv.get_shape().as_list()[1:]

replace_ip=re.compile(r'([0-9]+)(?:\.[0-9]+){3}',)
def text_to_wordlist(text, remove_stopwords=True, stem_words=False):
    # Clean the text, with the option to remove stopwords and to stem words.
    
    # Convert words to lower case and split them
    text = text.lower().split()
    
    
    
    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
    
    text = " ".join(text)
    
    
    #Replace IP address
    text=replace_ip.sub('',text)
    
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)


In [4]:
train_df = pd.read_csv(TRAIN_DATA_FILE)
test_df = pd.read_csv(TEST_DATA_FILE)

list_sentences_train = train_df["comment_text"].fillna("NA").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train_df[list_classes].values
list_sentences_test = test_df["comment_text"].fillna("NA").values
data=[]
for text in list_sentences_train:
    data.append(char2vec(text_to_wordlist(text)))
data=np.array(data)

test_data = []
for text in list_sentences_test:
    test_data.append(char2vec(text_to_wordlist(text)))
test_data=np.array(test_data)

In [5]:
test_data.shape

(153164, 1024)

In [6]:
def ConvolutionalBlock(input_shape, num_filters):
    model=Sequential()

    #1st conv layer
    model.add(Conv1D(filters = num_filters, kernel_size = 3, strides = 1, padding = "same", input_shape = input_shape))
    model.add(BatchNormalization())
    model.add(Activation("relu"))

    #2nd conv layer
    model.add(Conv1D(filters = num_filters, kernel_size = 3, strides = 1, padding = "same"))
    model.add(BatchNormalization())
    model.add(Activation("relu"))

    return model

#https://www.tensorflow.org/api_docs/python/tf/nn/top_k
def top_kmax(x):
    x=tf.transpose(x, [0, 2, 1])
    k_max = tf.nn.top_k(x, k=top_k)
    return tf.reshape(k_max[0], (-1, num_filters[-1]*top_k))

In [7]:
def vdcnn_model(num_filters, num_classes, sequence_max_length, num_chars, embedding_size, top_k, learning_rate=0.001):
    
    inputs=Input(shape=(sequence_max_length, ), dtype='int32', name='input')
    
    embedded_seq = Embedding(num_chars, embedding_size, input_length=sequence_max_length)(inputs)
    embedded_seq = BatchNormalization()(embedded_seq)
    #1st Layer
    conv = Conv1D(filters=64, kernel_size=3, strides=2, padding="same")(embedded_seq)
    
    #ConvBlocks
    for i in range(len(num_filters)):
        conv = ConvolutionalBlock(conv_shape(conv), num_filters[i])(conv)
        conv = MaxPooling1D(pool_size=3, strides=2, padding="same")(conv)
        
    def _top_k(x):
        x = tf.transpose(x, [0, 2, 1])
        k_max = tf.nn.top_k(x, k=top_k)
        return tf.reshape(k_max[0], (-1, num_filters[-1] * top_k))
    
    k_max = Lambda(_top_k, output_shape=(num_filters[-1] * top_k,))(conv)
    
    #fully connected layers
    # in original paper they didn't used dropouts
    fc1=Dense(512, activation='relu', kernel_initializer='he_normal')(k_max)
    fc1=Dropout(0.3)(fc1)
    fc2=Dense(512, activation='relu', kernel_initializer='he_normal')(fc1)
    fc2=Dropout(0.3)(fc2)
    out=Dense(num_classes, activation='sigmoid')(fc2)
    
    #optimizer
    #sgd = SGD(lr=learning_rate, decay=1e-6, momentum=0.9, nesterov=False)
    
    model = Model(inputs=inputs, outputs=out)
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [8]:
num_filters = [64, 128, 256, 512]
model=vdcnn_model(num_filters=num_filters, num_classes=6,num_chars=69, sequence_max_length=FEATURE_LEN,embedding_size=16,top_k=3)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 1024)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1024, 16)          1104      
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024, 16)          64        
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 512, 64)           3136      
_________________________________________________________________
sequential_1 (Sequential)    (None, 512, 64)           25216     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 256, 64)           0         
_________________________________________________________________
sequential_2 (Sequential)    (None, 256, 128)          75008     
__________

In [9]:
def limit_mem():
    import keras.backend as K
    K.get_session().close()
    cfg = K.tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    K.set_session(K.tf.Session(config=cfg))
    print('gpu memory cleaned')
    
def preds(k):
    from datetime import datetime
    y_temp = np.zeros((len(test_data), len(list_classes)))
    y_pred = np.zeros((len(test_data), len(list_classes)))
    i=0;
    from sklearn.model_selection import KFold
    kf = KFold(n_splits=k, random_state=2)

    for train_index, test_index in kf.split(data):
        limit_mem()
        start=datetime.now()
        print('fold====================>>>>>>>>>>',i+1)
        X_train, X_test = data[train_index], data[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model = None
        num_filters = [64, 128, 256, 512]
        model=vdcnn_model(num_filters=num_filters, num_classes=6,num_chars=69, sequence_max_length=FEATURE_LEN,embedding_size=16,top_k=3)

        early_stopping =EarlyStopping(monitor='val_loss', patience=5)
        bst_model_path = 'cv10_best_weights'+str(i+1) + '.h5'
        model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)

        hist = model.fit(X_train, y_train, \
                validation_data=(X_test, y_test), \
                epochs=200, batch_size=256,callbacks=[early_stopping, model_checkpoint])

        bst_val_score = min(hist.history['val_loss'])
        print('bst_val_score',bst_val_score)

        model.load_weights(bst_model_path)
        #model.fit(data, y,epochs=2, batch_size=256, shuffle=True,)
        
        y_temp = model.predict([test_data], batch_size=256, verbose=1)
        y_pred+=y_temp
        end=datetime.now()
        print(" ")
        print('time taken for this fold', end-start)
        i+=1
    y_test_pred=y_pred/k
    return y_test_pred

In [10]:
y_test=preds(10)

gpu memory cleaned
fold====================>>>>>>>>>> 1
Train on 143613 samples, validate on 15958 samples
Epoch 1/200
143613/143613 [==============================] - 265s 2ms/step - loss: 0.1147 - acc: 0.9710 - val_loss: 0.1356 - val_acc: 0.9603
Epoch 2/200
143613/143613 [==============================] - 237s 2ms/step - loss: 0.0678 - acc: 0.9784 - val_loss: 0.0707 - val_acc: 0.9788
Epoch 3/200
143613/143613 [==============================] - 214s 1ms/step - loss: 0.0618 - acc: 0.9796 - val_loss: 0.0643 - val_acc: 0.9798
Epoch 4/200
143613/143613 [==============================] - 214s 1ms/step - loss: 0.0578 - acc: 0.9803 - val_loss: 0.0590 - val_acc: 0.9798
Epoch 5/200
143613/143613 [==============================] - 213s 1ms/step - loss: 0.0552 - acc: 0.9809 - val_loss: 0.0596 - val_acc: 0.9806
Epoch 6/200
143613/143613 [==============================] - 213s 1ms/step - loss: 0.0535 - acc: 0.9813 - val_loss: 0.0660 - val_acc: 0.9797
Epoch 7/200
143613/143613 [====================

Epoch 11/200
143614/143614 [==============================] - 213s 1ms/step - loss: 0.0473 - acc: 0.9826 - val_loss: 0.0856 - val_acc: 0.9795
Epoch 12/200
143614/143614 [==============================] - 213s 1ms/step - loss: 0.0465 - acc: 0.9828 - val_loss: 0.0681 - val_acc: 0.9805
Epoch 13/200
143614/143614 [==============================] - 213s 1ms/step - loss: 0.0457 - acc: 0.9829 - val_loss: 0.1153 - val_acc: 0.9796
Epoch 14/200
143614/143614 [==============================] - 213s 1ms/step - loss: 0.0451 - acc: 0.9831 - val_loss: 0.0615 - val_acc: 0.9802
bst_val_score 0.056414422745740364
153164/153164 [==============================] - 82s 533us/step
 
time taken for this fold 0:51:33.760633
gpu memory cleaned
fold====================>>>>>>>>>> 5
Train on 143614 samples, validate on 15957 samples
Epoch 1/200
143614/143614 [==============================] - 231s 2ms/step - loss: 0.1124 - acc: 0.9708 - val_loss: 0.1134 - val_acc: 0.9756
Epoch 2/200
143614/143614 [================

Train on 143614 samples, validate on 15957 samples
Epoch 1/200
143614/143614 [==============================] - 243s 2ms/step - loss: 0.1110 - acc: 0.9708 - val_loss: 0.0704 - val_acc: 0.9780
Epoch 2/200
143614/143614 [==============================] - 215s 1ms/step - loss: 0.0685 - acc: 0.9782 - val_loss: 0.0712 - val_acc: 0.9791
Epoch 3/200
143614/143614 [==============================] - 216s 2ms/step - loss: 0.0618 - acc: 0.9796 - val_loss: 0.0682 - val_acc: 0.9774
Epoch 4/200
143614/143614 [==============================] - 215s 1ms/step - loss: 0.0589 - acc: 0.9802 - val_loss: 0.0726 - val_acc: 0.9800
Epoch 5/200
143614/143614 [==============================] - 215s 1ms/step - loss: 0.0563 - acc: 0.9808 - val_loss: 0.0551 - val_acc: 0.9812
Epoch 6/200
143614/143614 [==============================] - 215s 1ms/step - loss: 0.0541 - acc: 0.9810 - val_loss: 0.0575 - val_acc: 0.9808
Epoch 7/200
143614/143614 [==============================] - 215s 1ms/step - loss: 0.0526 - acc: 0.9814